# Iterative pruning pipeline
Model: Multi Layer Perceptron

Pruning Mehtods: global and local iterative magnitude pruning

*Pruning functions as class methods*

In [1]:
EXPERIMENT_NAME = 'mlp-local-magnitude-unstruct'


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import json
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Prune, Train Attack Pipeline

In [11]:
loc_pruning_pgd_success_rates = []
loc_pruning_cw_success_rates = []

loc_pruning_all_accuracies = []

for j in tqdm(range(3)):
    model_for_loc_pruning = initialize_base_model(j, save_weights=True)
    loc_accuracies = []
    loc_pgd_success_rate = []
    loc_cw_success_rate = []
    compression_rates = [1, 2, 4, 8, 16, 32, 64]
    pruning_ratios = [1-1/x for x in compression_rates]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model_for_loc_pruning.load_weights(f'./saved-weights/{EXPERIMENT_NAME}-{j}')
        
        #iteratively prune and train (only to convergence if the final stage of pruning is reached)
        for i in range(index + 1):
            if i != index:
                
                #local pruning
                model_for_loc_pruning.prune_locally(pruning_ratio)
                model_for_loc_pruning.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                              metrics=['accuracy'],
                             )
                #fine-tune
                model_for_loc_pruning = train_model(model_for_loc_pruning, to_convergence=False)
            if i == index:
                print('final pruning and eval')

                model_for_loc_pruning.prune_globally(pruning_ratio)
                model_for_loc_pruning.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                              metrics=['accuracy'],
                             )
                model_for_loc_pruning = train_model(model_for_loc_pruning, to_convergence=True)
                loc_accuracies.append(model_for_loc_pruning.evaluate(x_test, y_test, verbose=0))
                loc_pgd_success_rate.append(pgd_attack(model_for_loc_pruning))
                #loc_cw_success_rate.append(cw2_attack(model_for_loc_pruning))

                
    loc_pruning_all_accuracies.append(loc_accuracies)
    loc_pruning_pgd_success_rates.append(loc_pgd_success_rate)
    loc_pruning_cw_success_rates.append(loc_cw_success_rate)



#write to csv and json
pd.DataFrame(loc_pruning_all_accuracies).to_csv(f'saved-results/{EXPERIMENT_NAME}-accuracies.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-accuracies.json', 'w') as f:
    json.dump(loc_pruning_all_accuracies, f)
    
pd.DataFrame(loc_pruning_pgd_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-pgd-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-pgd-success.json', 'w') as f:
    json.dump(loc_pruning_pgd_success_rates, f)
    
pd.DataFrame(loc_pruning_cw_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-cw2-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-cw2-success.json', 'w') as f:
    json.dump(loc_pruning_cw_success_rates, f)


  0%|          | 0/3 [00:00<?, ?it/s]

938/938 [==============================] - 4s 5ms/step - loss: 1.5764 - accuracy: 0.8974 - val_loss: 1.5207 - val_accuracy: 0.9433



0it [00:00, ?it/s]

final pruning and eval
Epoch 1/100
938/938 [==============================] - 4s 4ms/step - loss: 1.5111 - accuracy: 0.9523 - val_loss: 1.5030 - val_accuracy: 0.9600
Epoch 2/100
938/938 [==============================] - 4s 5ms/step - loss: 1.4978 - accuracy: 0.9650 - val_loss: 1.4986 - val_accuracy: 0.9631
Epoch 3/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4910 - accuracy: 0.9716 - val_loss: 1.4927 - val_accuracy: 0.9697
Epoch 4/100
938/938 [==============================] - 3s 4ms/step - loss: 1.4862 - accuracy: 0.9760 - val_loss: 1.4936 - val_accuracy: 0.9683
Epoch 5/100
938/938 [==============================] - 4s 5ms/step - loss: 1.4826 - accuracy: 0.9794 - val_loss: 1.4865 - val_accuracy: 0.9752
Epoch 6/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4806 - accuracy: 0.9812 - val_loss: 1.4860 - val_accuracy: 0.9753
Epoch 7/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4791 - accuracy: 0.9827 - val_loss: 1.4867


1it [01:33, 93.51s/it]

...


5it [05:53, 76.94s/it]

Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.7946 - accuracy: 0.8214 - val_loss: 1.5824 - val_accuracy: 0.9054
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 1.5663 - accuracy: 0.9152 - val_loss: 1.5532 - val_accuracy: 0.9212
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.5457 - accuracy: 0.9282 - val_loss: 1.5417 - val_accuracy: 0.9282
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 1.5358 - accuracy: 0.9352 - val_loss: 1.5356 - val_accuracy: 0.9325
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.5294 - accuracy: 0.9399 - val_loss: 1.5312 - val_accuracy: 0.9357
Epoch 2/2
938/938 [==============================] - 4s 5ms/step - loss: 1.5250 - accuracy: 0.9431 - val_loss: 1.5280 - val_accuracy: 0.9387
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.5214 - accuracy: 0.9462 - val_loss: 1.5250 - val_accuracy: 0.9402
Epoch 2/2
938


6it [09:23, 116.68s/it]

Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 2.2225 - accuracy: 0.0987 - val_loss: 2.1529 - val_accuracy: 0.0980
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 2.1399 - accuracy: 0.0987 - val_loss: 2.1273 - val_accuracy: 0.0980
Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 2.1238 - accuracy: 0.0987 - val_loss: 2.1179 - val_accuracy: 0.0980
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 2.1167 - accuracy: 0.0987 - val_loss: 2.1129 - val_accuracy: 0.0980
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 2.1128 - accuracy: 0.0987 - val_loss: 2.1102 - val_accuracy: 0.0980
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 2.1103 - accuracy: 0.0987 - val_loss: 2.1090 - val_accuracy: 0.0980
Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 2.1086 - accuracy: 0.0987 - val_loss: 2.1078 - val_accuracy: 0.0980
Epoch 2/2
938


7it [11:43, 100.50s/it]
100%|██████████| 3/3 [35:06<00:00, 702.25s/it]


# Helper Functions

In [9]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model



def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 2) or (index == 4):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index].flatten()
            mask = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            #mask_df = pd.DataFrame(mask)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                mask[idx_to_delete] = 0
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            mask_reshaped = mask.reshape(dims)
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+1] = mask_reshaped
            weights_to_prune[index] = weights_reshaped
    
    return weights_to_prune



def pgd_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.LinfProjectedGradientDescentAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def cw2_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.L2CarliniWagnerAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[.5]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def initialize_base_model(index, save_weights=False):
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=1,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    if save_weights == True:
        model.save_weights(f'./saved-weights/{EXPERIMENT_NAME}-{index}')
    return model
    

# Load Data

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:500].reshape(500,28*28))
y = tf.convert_to_tensor([y_train[:500]])[0];

# Define Model

In [7]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')

        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        x = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, x)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
    def prune_globally(self,ratio):
                
        shape1 = self.dense1.w.shape
        shape2 = self.dense2.w.shape
        shape3 = self.dense3.w.shape

        flat_weights = np.append(self.dense1.w.numpy().flatten() ,self.dense2.w.numpy().flatten())
        flat_weights = np.append(flat_weights ,self.dense3.w.numpy().flatten())
        flat_mask = np.append(self.dense1.mask.numpy().flatten(), self.dense2.mask.numpy().flatten())
        flat_mask = np.append(flat_mask, self.dense3.mask.numpy().flatten())
        
        no_of_weights_to_prune = int(len(flat_weights)*ratio)
        indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
        
        for idx_to_delete in indices_to_delete:
            flat_mask[idx_to_delete] = 0
            flat_weights[idx_to_delete] = 0
            
        w1 = flat_weights[:shape1[0]*shape1[1]].reshape(shape1)
        w2 = flat_weights[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        w3 = flat_weights[-shape3[0]*shape3[1]:].reshape(shape3)
        m1 = flat_mask[:shape1[0]*shape1[1]].reshape(shape1)
        m2 = flat_mask[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        m3 = flat_mask[-shape3[0]*shape3[1]:].reshape(shape3)
        self.set_weights([w1,m1,w2,m2,w3,m3])
        #print(weights)
        return
    
    def prune_locally(self, ratio):
        layers = self.get_weights()
        for index, weights in enumerate(layers):
            if (index == 0) or (index == 2) or (index == 4):
                shape = weights.shape
                flat_weights = weights.flatten()
                mask = layers[index+1].flatten()
                
                no_of_weights_to_prune = int(len(flat_weights)*ratio)
                indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                
                mask_reshaped = mask.reshape(shape)
                weights_reshaped = flat_weights.reshape(shape)
                layers[index+1] = mask_reshaped
                layers[index] = weights_reshaped
        self.set_weights(layers)
        return 

# Compile and Train Model

In [ ]:
model = LeNet300_100()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=1,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )

model.save('./saved-models/mini-pipeline-mlp-baseline-model')
model.save_weights('./saved-models/weights')